In [1]:
!pip install -q /pfss/mlde/workspaces/mlde_wsp_MorphPiece/git/MorphPiece transformers datasets

DEPRECATION: devscripts 2.22.1ubuntu1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of devscripts or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.17.0a0 requires torch==2.3.0a0+40ec155e58, but you have torch 2.3.0a0+40ec155e58.nv24.3 which is incompatible.


In [2]:
from datasets import load_dataset, DatasetDict
from morphpiece import MorphPiece
from transformers import AutoTokenizer

In [3]:
# raw_datasets = load_dataset('wikitext','wikitext-2-raw-v1')
raw_datasets = load_dataset('parquet',data_dir="/pfss/mlde/workspaces/mlde_wsp_MorphPiece/data/fineweb-edu/sample/10BT",split='train').select_columns(['id','text'])

Loading dataset shards:   0%|          | 0/98 [00:00<?, ?it/s]

In [4]:
raw_datasets = DatasetDict(raw_datasets.train_test_split(test_size=0.001))

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'text'],
        num_rows: 9662428
    })
    test: Dataset({
        features: ['id', 'text'],
        num_rows: 9673
    })
})

In [6]:
raw_datasets['train'][10]

{'id': '<urn:uuid:2465584b-9763-4d17-8dc1-49dce228e762>',
 'text': 'Mt Kelimutu National Park is the smallest among six national parks in the stretch between the Nusa Tenggara islands. Its size obviously does not matter much when it offers one of the most spectacular wonders that nature has to offer. There are three lakes on the mountain sharing the same name, Kelimutu, meaning ‘the boiling lake’. Each has its own colors and a local name. But all are believed to be the resting place of departed souls.\nKelimutu lakes are unpredictable as to when and what color they will change into. Sometimes, the colors are blue, green, and black, and some other times they turn to white, red, and blue. The last time a traveler saw them, one was dark brown, just like a pond of chocolate. Previously, the east lake was red and the middle lake was blue.\nLake Kelimutu is part of the Kelimutu National Park, presenting a highly aesthetic value and surreal experience. The national park’s highest point is 5,6

In [7]:
m = MorphPiece(data_dir='/pfss/mlde/workspaces/mlde_wsp_MorphPiece/data/tokenizers_50k/gpt2-owt-morph15k')

In [8]:
b = AutoTokenizer.from_pretrained("gpt2")

In [9]:
text_column_name='text'
def tokenize_function(examples,tokenizer):
    return tokenizer(examples[text_column_name],return_attention_mask=False)

In [10]:
tokenizer = m

In [12]:
tokenized_datasets = raw_datasets['train'].select(range(10000)).map(
            tokenize_function, 
            fn_kwargs={'tokenizer':tokenizer},
            batched=True,
            num_proc=1,
            remove_columns=raw_datasets['train'].column_names,
        )

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets

Dataset({
    features: ['input_ids'],
    num_rows: 10000
})

In [ ]:
block_size=2048

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

In [15]:
# tokenized_datasets = tokenized_datasets.remove_columns('attention_mask')
tokenized_datasets

Dataset({
    features: ['input_ids'],
    num_rows: 10000
})

In [ ]:
lm_datasets = tokenized_datasets.map(
            group_texts,
            batched=True,
            num_proc=16,
        )

Map (num_proc=16):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
lm_datasets

NameError: name 'lm_datasets' is not defined

In [ ]:
lm_datasets.save_to_disk('/pfss/mlde/workspaces/mlde_wsp_MorphPiece/data/fineweb-edu/grouped_owt_morph15k_50k_2048')

Saving the dataset (0/1 shards):   0%|          | 0/57015 [00:00<?, ? examples/s]

In [30]:
lm_datasets.num_rows/lm_datasets_b.num_rows

1.1432266602502406